# loss가 가장 낮게 나올 때의 하이퍼 파라미터를 저장하는 코드
저장하는 이유: 매번 학습하면 시간 오래걸리니까
<br>
LSH 07 Best Model Saving

In [1]:
import numpy as np, os
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# print(gpus)
# if gpus:  # gpu가 있다면, 용량 한도를 5GB로 설정
#     tf.config.experimental.set_virtual_device_configuration(gpus[0], 
#                                                             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
#GPU 사용 설정, -1이면 CPU 사용
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
! ps

    PID TTY          TIME CMD
 232850 pts/5    00:00:00 ps


In [3]:
# loss가 순간순간 낮아지면 그때그때의 하이퍼파라미터로 된 LSTM(모델 자체)을 저장하는 코드
# loss가 가장 낮은것만 빼고 다 지울 것. (loss는 파일확장자 바로 앞 숫자임.)


import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 42
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 42 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2]))) # 모델에는 1,2번째만 들어가니까 데이터의 shape을 바꿀 때 0번째의 크기는 상관이 없음.
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'seed42-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=50, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

Epoch 1/500
37/37 [==============================] - 5s 66ms/step - loss: 0.6757 - acc: 0.5984 - val_loss: 0.6655 - val_acc: 0.6177

Epoch 00001: val_loss improved from inf to 0.66548, saving model to ./model/seed42-01-0.6655.hdf5
Epoch 2/500
37/37 [==============================] - 2s 54ms/step - loss: 0.6691 - acc: 0.6089 - val_loss: 0.6488 - val_acc: 0.6177

Epoch 00002: val_loss improved from 0.66548 to 0.64878, saving model to ./model/seed42-02-0.6488.hdf5
Epoch 3/500
37/37 [==============================] - 2s 54ms/step - loss: 0.6012 - acc: 0.6711 - val_loss: 0.5525 - val_acc: 0.7342

Epoch 00003: val_loss improved from 0.64878 to 0.55255, saving model to ./model/seed42-03-0.5525.hdf5
Epoch 4/500
37/37 [==============================] - 2s 52ms/step - loss: 0.4873 - acc: 0.7783 - val_loss: 0.5084 - val_acc: 0.7607

Epoch 00004: val_loss improved from 0.55255 to 0.50836, saving model to ./model/seed42-04-0.5084.hdf5
Epoch 5/500
37/37 [==============================] - 2s 53ms/ste

37/37 [==============================] - 2s 53ms/step - loss: 0.1265 - acc: 0.9631 - val_loss: 0.8770 - val_acc: 0.7393

Epoch 00044: val_loss did not improve from 0.50288
Epoch 45/500
37/37 [==============================] - 2s 53ms/step - loss: 0.0882 - acc: 0.9793 - val_loss: 0.9463 - val_acc: 0.7329

Epoch 00045: val_loss did not improve from 0.50288
Epoch 46/500
37/37 [==============================] - 2s 53ms/step - loss: 0.0772 - acc: 0.9830 - val_loss: 0.9864 - val_acc: 0.7354

Epoch 00046: val_loss did not improve from 0.50288
Epoch 47/500
37/37 [==============================] - 2s 53ms/step - loss: 0.0676 - acc: 0.9853 - val_loss: 1.0200 - val_acc: 0.7354

Epoch 00047: val_loss did not improve from 0.50288
Epoch 48/500
37/37 [==============================] - 2s 53ms/step - loss: 0.0651 - acc: 0.9853 - val_loss: 1.0468 - val_acc: 0.7322

Epoch 00048: val_loss did not improve from 0.50288
Epoch 49/500
37/37 [==============================] - 2s 53ms/step - loss: 0.0634 - acc:

In [6]:
# best model의 정확도를 확인하는 코드
from keras.models import load_model
best_model = load_model('./model/seed42-05-0.5021.hdf5') 

dic_42={}
for seed in range(0, 50):
    random.seed(seed) # random.seed는 seed setting을 위한 함수임.

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic_42[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')

Using TensorFlow backend.


정확도 :0.8194174757281554, seed_num = 0
정확도 :0.8362459546925566, seed_num = 1
정확도 :0.8207119741100324, seed_num = 2
정확도 :0.8388349514563107, seed_num = 3
정확도 :0.8252427184466019, seed_num = 4
정확도 :0.8401294498381877, seed_num = 5
정확도 :0.8103559870550162, seed_num = 6
정확도 :0.8155339805825242, seed_num = 7
정확도 :0.827831715210356, seed_num = 8
정확도 :0.8317152103559871, seed_num = 9
정확도 :0.8265372168284789, seed_num = 10
정확도 :0.8310679611650486, seed_num = 11
정확도 :0.8349514563106796, seed_num = 12
정확도 :0.8343042071197411, seed_num = 13
정확도 :0.8381877022653722, seed_num = 14
정확도 :0.8336569579288026, seed_num = 15
정확도 :0.8330097087378641, seed_num = 16
정확도 :0.8174757281553398, seed_num = 17
정확도 :0.8271844660194175, seed_num = 18
정확도 :0.8388349514563107, seed_num = 19
정확도 :0.8284789644012945, seed_num = 20
정확도 :0.8142394822006472, seed_num = 21
정확도 :0.8226537216828479, seed_num = 22
정확도 :0.8090614886731392, seed_num = 23
정확도 :0.8317152103559871, seed_num = 24
정확도 :0.8466019417475729, seed_num = 

In [7]:
# seed = 42의 정확도 df 만들고 평균 확인 => 82.6
df_42 = pd.DataFrame.from_dict(dic_42, orient='index')
df_42.mean()

0    0.826731
dtype: float64

# seed별 best model 저장
seed 수동으로 바꿔줄 것

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics 
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


import random
# ----------------------
seed_num = 50
# ----------------------
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

# ---------------------
seed_num = 50 
# ---------------------
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))

from tensorflow.keras.callbacks import ModelCheckpoint
import os

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'seed'f'{seed_num}''-{epoch:02d}-{val_loss:.4f}.hdf5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), loss = "binary_crossentropy", metrics=['acc'])
lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=500,  callbacks=[early_stop,cb_checkpoint], shuffle=False)

In [ ]:
# ----------------------------
seed_num = 50
# ----------------------------

from keras.models import load_model
best_model = load_model('./model/seed50-04-0.5157.hdf5') # 수동으로 확인

dic={}
for seed in range(0, 50):
    random.seed(seed)

    x = np.load('/project/LSH/x_(7727,10,4068).npy')
    y = np.load('/project/LSH/y_(7727,1).npy')

    idx = list(range(len(x)))
    random.shuffle(idx)

    i = round(x.shape[0]*0.8)
    X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
    X_test, y_test = x[idx[i:],:,:], y[idx[i:]]
    
    pred = best_model.predict(X_test)
    pred[pred>0.5]=1
    pred[pred<=0.5]=0
    acc = metrics.accuracy_score(y_test, pred)
    dic[seed]=acc
    print(f'정확도 :{metrics.accuracy_score(y_test, pred)}, seed_num = {seed}')
    
df = pd.DataFrame.from_dict(dic, orient='index')
print(f'seed = {seed_num}의 정확도 df 만들고 평균 확인 : {df.mean().values}')